<a href="https://colab.research.google.com/github/anuppohekar/Deep-Learning-Projects/blob/main/Copy_of_FINETUNING_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LIBRARIES

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

#INCLUDE MODEL

In [ ]:
from keras.layers import Input
oldmodel = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(32,32,3)))

#ADD NEW TOP LAYER / ADD NEW FINAL LAYER

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

topmodel = Sequential()
topmodel.add(Flatten())
topmodel.add(Dense(units=128, activation='relu'))
topmodel.add(Dense(units=10, activation='softmax'))

#COMBINE OLD MODEL WITH NEW TOP LAYER

In [ ]:
from keras.models import Model 
finalmodel = Model(inputs = oldmodel.input, outputs = topmodel(oldmodel.output))

#MAKE  OLD MODEL NON TRAINABLE

In [ ]:
for layer in oldmodel.layers:
  layer.trainable=False

In [ ]:
finalmodel.summary()

#COMPILE AND FIT

In [ ]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#CALLBACKS

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
es = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
filepath="bestweight.h5"
mc = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
rd = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto')

In [ ]:
import keras
opt = keras.optimizers.Adam(learning_rate=1e-4)
finalmodel.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'], callbacks=[es, mc, rd])

finalmodel.fit(x=x_train, y=y_train, epochs=10, validation_split=0.2)

In [ ]:
finalmodel.evaluate(x_test, y_test)

#Check pretrained model versus fine tuned model

In [ ]:
from keras.layers import Input
pre_trained_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(32,32,3)))

#Add new top layer

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

topmodel = Sequential()
topmodel.add(Flatten())
topmodel.add(Dense(units=10, activation='softmax'))

#COMBINE OLD MODEL WITH NEW TOP LAYER

In [ ]:
from keras.models import Model 
finalmodel2 = Model(inputs = pre_trained_model.input, outputs = topmodel(pre_trained_model.output))

In [ ]:
finalmodel2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
finalmodel2.evaluate(x_test, y_test)